In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Change working directory to be current folder
import os
os.chdir('/content/gdrive/My Drive/Practice Module/Violence-Detection')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import cv2
import h5py
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Activation, Conv3D, Dense, Dropout, LSTM
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

print('tensorflow version :', tf.__version__)

In [ ]:
class VideotoData:

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def get_data(self, filename):
      
        cap = cv2.VideoCapture(filename)
        nframes = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        framearray = []

        for i in range(int(nframes)):
            ret, frame = cap.read()
            frame = cv2.resize(frame, (self.height, self.width))
            framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()

        return np.array(framearray)

In [ ]:
def videoto3D(depth, videoarray):

    vid3D = []
    nframes = videoarray.shape[0]

    if (nframes>=depth):
        frames = [x * nframes / depth for x in range(depth)]
    else:
        frames = [x for x in range(int(nframes))]

    for i in range(len(frames)):
        vid3D.append(videoarray[int(frames[i]), :, :])
    
    return np.array(vid3D)

In [ ]:
# Define parameter setting
class Args:
    batch = 128
    epoch = 50
    nclass = 2 # 11 action categories
    depth = 10
    rows = 32
    cols = 32
    skip = True # Skip: randomly extract frames; otherwise, extract first few frames

param_setting = Args()
img_rows = param_setting.rows
img_cols = param_setting.cols
frames = param_setting.depth
nb_classes = param_setting.nclass
channel = 1

In [ ]:
category = {
  1: "Violence",
  0: "No Violence",
}

In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/Practice Module/Violence-Detection'

In [ ]:
def inference_violence(video_frames, model_path, weights_path, videoto3D):

    category = {
                1: "Violence",
                0: "No Violence",
               }
    c3D_model = load_model(os.path.join(os.getcwd(),model_path))
    c3D_model.load_weights(os.path.join(os.getcwd(),weights_path))
    vid3D = videoto3D(10, video_frames)
    vid3D = np.expand_dims(vid3D, axis=3)
    vid3D = vid3D.transpose((1,2,0,3))
    vid3D = np.expand_dims(vid3D, axis=0)
    
    y_pred = c3D_model.predict(vid3D)
  
    return category[np.argmax(y_pred)]


In [ ]:
filename = '../RWF-2000/val/no34.avi'
vidData = VideotoData(32,32)
video_frames = vidData.get_data(filename)
video_frames.shape

(150, 32, 32)

In [ ]:
model_path   = 'c3d_model.h5'
weights_path = 'c3d_model_weights.h5'
action = inference_violence(video_frames, model_path, weights_path, videoto3D)
print(action)

No Violence


In [ ]:
## Accuracy, Precision and Recall from models trained

In [ ]:
# datasets1 = ["datasets/c3d_val_150f.npz", "datasets/c3d_val_120f.npz", "datasets/c3d_val_90f.npz", "datasets/c3d_val_60f.npz",]
# datasets2 = ["datasets/c3d_opt_flow_val.npz", "datasets/c3d_keypoints_val.npz"]
with np.load("datasets/c3d_val_150f.npz") as npzfile:
    x_test = npzfile["X"]
    y_test = npzfile["Y"] 
print(x_test.shape, y_test.shape) 

(400, 32, 32, 10, 1) (400, 2)


In [ ]:
model_path = 'c3d_model.hdf5'
#weights_path = 'c3d_model_150f_weights.h5'
weights_paths1 = ['c3d_model_150f_weights.h5', 'c3d_model_120f_weights.h5', 'c3d_model_90f_weights.h5', 'c3d_model_60f_weights.hdf5']
for weights_path in weights_paths1:
      
    c3D_model = load_model(os.path.join(os.getcwd(),model_path))
    c3D_model.load_weights(os.path.join(os.getcwd(),weights_path))
    y_pred = c3D_model.predict(x_test, verbose=0)
    print("Confusion matrix")
    print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
    print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
    print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
    print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Confusion matrix
[[149  51]
 [ 35 165]]
Accuracy  : 0.785
Precision : 0.7638888888888888
Recall    : 0.825
Confusion matrix
[[149  51]
 [ 55 145]]
Accuracy  : 0.735
Precision : 0.7397959183673469
Recall    : 0.725
Confusion matrix
[[139  61]
 [ 43 157]]
Accuracy  : 0.74
Precision : 0.7201834862385321
Recall    : 0.785
Confusion matrix
[[145  55]
 [ 41 159]]
Accuracy  : 0.76
Precision : 0.7429906542056075
Recall    : 0.795


In [ ]:
with np.load("datasets/c3d_val_150f.npz") as npzfile:
    x_test = npzfile["X"]
    y_test = npzfile["Y"] 
print(x_test.shape, y_test.shape)

c3D_model = load_model(os.path.join(os.getcwd(),'c3d_model.hdf5'))
weights_path = 'c3d_model_120f_5fold_weights.hdf5'
c3D_model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = c3D_model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

(400, 32, 32, 10, 1) (400, 2)
Confusion matrix
[[155  45]
 [ 67 133]]
Accuracy  : 0.72
Precision : 0.7471910112359551
Recall    : 0.665


In [ ]:
with np.load("datasets/c3d_opt_flow_val.npz") as npzfile:
    x_test = npzfile["X"]
    y_test = npzfile["Y"] 
print(x_test.shape, y_test.shape)

weights_path = 'c3d_model_opt_flow_weights.h5'
c3D_model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = c3D_model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))


(222, 32, 32, 10, 1) (222, 2)
Confusion matrix
[[51 60]
 [41 70]]
Accuracy  : 0.545045045045045
Precision : 0.5384615384615384
Recall    : 0.6306306306306306


In [ ]:
with np.load("datasets/c3d_keypoints_val.npz") as npzfile:
    x_test = npzfile["X"]
    y_test = npzfile["Y"] 
print(x_test.shape, y_test.shape)

weights_path = 'c3d_model_keypoints_weights.h5'
c3D_model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = c3D_model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

(222, 32, 32, 10, 1) (222, 2)
Confusion matrix
[[63 48]
 [18 93]]
Accuracy  : 0.7027027027027027
Precision : 0.6595744680851063
Recall    : 0.8378378378378378


In [ ]:
chunk_size = 4096
n_chunks = 100
rnn_size = 512
model_name = 'model_lstm_keypoints'

model = Sequential()
model.add(LSTM(rnn_size, input_shape=(n_chunks, chunk_size)))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
def process_data(h5_file):
    
    #joint_transfer=[]
    frames_num=100
    count = 0
    data, target = [], []

    with h5py.File(h5_file, 'r') as f:
            
        X_batch = f['data'][:]
        y_batch = f['labels'][:]

    for i in range(int(len(X_batch)/frames_num)):
        inc = count+frames_num
        data.append(X_batch[count:inc])
        target.append(y_batch[count])
        count =inc

    data, target = np.array(data), np.array(target)
            
    return data, target

In [ ]:
x_test, y_test = process_data('datasets/lstm_rgb_val.h5')
print(x_test.shape, y_test.shape)

(400, 100, 4096) (400, 2)


In [ ]:
x_test, y_test = process_data('datasets/lstm_rgb_val.h5')
print(x_test.shape, y_test.shape)
model = load_model('lstm_model.h5')
weights_path = 'model_lstm_rgb_weights.h5'
model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))


Confusion matrix
[[156  44]
 [ 77 123]]
Accuracy  : 0.6975
Precision : 0.7365269461077845
Recall    : 0.615


In [ ]:
model = load_model('lstm_model.h5')
x_test, y_test = process_data('datasets/lstm_opt_flow_val.h5')
weights_path = 'model_lstm_opt_flow_weights.h5'
model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Confusion matrix
[[184   0]
 [111   0]]
Accuracy  : 0.6237288135593221
Precision : 0.0
Recall    : 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = load_model('lstm_model.h5')
x_test, y_test = process_data('datasets/lstm_keypoints_val.h5')
weights_path = 'model_lstm_keypoints_weights.h5'
model.load_weights(os.path.join(os.getcwd(),weights_path))
y_pred = model.predict(x_test, verbose=0)
print("Confusion matrix")
print(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Accuracy  :', accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Precision :', precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))
print('Recall    :', recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Confusion matrix
[[184   0]
 [111   0]]
Accuracy  : 0.6237288135593221
Precision : 0.0
Recall    : 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
